<a href="https://colab.research.google.com/github/prateekjoshi565/textrank_text_summarization/blob/master/TestRank_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
%%capture

!wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
!unzip 'master.zip'

In [3]:
%%capture
# Install the vncorenlp python wrapper
!pip install vncorenlp
# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

In [4]:
import glob
import pandas as pd
import concurrent.futures
from datasets import *

In [70]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('/content/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('/content/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('/content/vietnews-master/data/test_tokenized/*')

In [71]:
def read_content(pathfile):
  """
  Input: Path of txt file
  Output: A dictionary has keys 'original' and 'summary'
  """
  with open(pathfile) as f:
    rows  = f.readlines()
    original = ' '.join(''.join(rows[4:]).split('\n'))
    summary = ' '.join(rows[2].split('\n'))
          
  return {'file' : pathfile,
          'original': original, 
          'summary': summary}

In [72]:
read_content(train_paths[0])

{'file': '/content/vietnews-master/data/train_tokenized/035473.txt.seg',
 'original': 'Một người_dân từng sống ở khu_vực Moon_Bay_Circle , Wellington thuộc hạt Palm_Beach của bang Florida hồi giữa tháng 8 kiểm_tra hình_ảnh vệ_tinh trên Google_Maps và tình_cờ phát_hiện một chiếc ô_tô bị chìm trong hồ nước tại đó . Người này thông_báo cho một cư_dân đang sống tại Moon_Bay_Circle để kiểm_tra . Sau khi dùng flycam để chắc_chắn đó là một chiếc ôtô bị chìm dưới hồ , người này gọi điện_báo nhà_chức_trách , sở cảnh_sát hạt Palm_Beach hôm 12/9 cho_hay . Cảnh_sát ngày 28/8 tới hồ nước trục_vớt chiếc xe và tìm thấy một hài_cốt bên trong . Một tuần sau , hài_cốt được xác_định là của William_Moldt , người mất_tích từ năm 1997 . Theo Charley_Project , một cơ_sở_dữ_liệu trực_tuyến về các trường_hợp mất_tích ở Mỹ , Google_Maps đã chụp ảnh chiếc xe bị chìm dưới hồ này từ năm 2007 , nhưng dường_như không ai nhận_ra nó trong hơn 10 năm qua . Ảnh vệ_tinh cho thấy rõ hình_dạng chiếc ôtô trong lòng hồ ở Pal

In [73]:
def get_dataframe(pathfiles):
  with concurrent.futures.ProcessPoolExecutor() as executor:
    data = executor.map(read_content, pathfiles)
  
  # Make blank dataframe
  data_df = list()
  for d in data:
    data_df.append(d)
  data_df = pd.DataFrame(data_df)
  data_df.dropna(inplace = True)
  data_df = data_df.sample(frac=1).reset_index(drop=True)

  return data_df

In [74]:
train_df = get_dataframe(train_paths)

In [75]:
test_df = get_dataframe(test_paths)

In [76]:
train_df.head(2)

,file,original,summary
0,/content/vietnews-master/data/train_tokenized/...,"Hôn_lễ của Harry , 33 tuổi , và Meghan , 36 tu...",Hoàng tử Anh Harry và nữ diễn viên Mỹ Me...
1,/content/vietnews-master/data/train_tokenized/...,"Phát_biểu tại lễ kỷ_niệm , ông Trịnh_Văn_Chiến...","Tối 8-5 , tại quảng_trường Lam_Sơn , TP Thanh_..."


In [77]:
train_df.original[0]

'Hôn_lễ của Harry , 33 tuổi , và Meghan , 36 tuổi , diễn ra tại lâu_đài Windsor của hoàng_gia Anh , thuộc thị_trấn Windsor , cách thủ_đô London hơn 30 km . Họ đã phá vỡ truyền_thống khi chọn tổ_chức đám_cưới vào cuối tuần , trong khi các đám_cưới trước_đây của hoàng_gia đều diễn ra vào ngày trong tuần , Telegraph cho_hay . Hoàng_tử Harry và hôn thê Meghan_Markle . Ảnh : Kengsington_Palace Đúng 12h trưa ( 18 h giờ Hà_Nội ) , cô_dâu và chú_rể sẽ làm lễ tại nhà_nguyện St_George bên trong lâu đài trước sự chứng_kiến của khoảng 800 khách mời . Do bố của Meghan vừa trải qua ca mổ tim , Thái_tử Charles , bố của Harry , sẽ là người dẫn cô_dâu vào lễ đường . Một tiếng sau , họ sẽ cùng lên cỗ xe_ngựa để tham_gia lễ rước quanh toà nhà và chào dân_chúng . 2.640 khách mời là dân_thường do cặp đôi lựa_chọn kỹ_lưỡng sẽ được chứng_kiến nghi_lễ này từ sân lâu đài , trong đó có hàng trăm cá_nhân đến từ các tổ_chức từ_thiện . Các biện_pháp bảo_vệ an_ninh cho đám_cưới Sau lễ rước , Nữ_hoàng Elizabeth sẽ c

In [82]:
# split the the text in the articles into sentences
sentences = []
for s in train_df.original.head(2):
  sentences.append(sent_tokenize(s))  

In [ ]:
sentences

In [80]:
# bỏ qua cái này 
# flatten the list
# sentences = [y for x in sentences[0] for y in x]

In [84]:
sentences[0]

['Hôn_lễ của Harry , 33 tuổi , và Meghan , 36 tuổi , diễn ra tại lâu_đài Windsor của hoàng_gia Anh , thuộc thị_trấn Windsor , cách thủ_đô London hơn 30 km .',
 'Họ đã phá vỡ truyền_thống khi chọn tổ_chức đám_cưới vào cuối tuần , trong khi các đám_cưới trước_đây của hoàng_gia đều diễn ra vào ngày trong tuần , Telegraph cho_hay .',
 'Hoàng_tử Harry và hôn thê Meghan_Markle .',
 'Ảnh : Kengsington_Palace Đúng 12h trưa ( 18 h giờ Hà_Nội ) , cô_dâu và chú_rể sẽ làm lễ tại nhà_nguyện St_George bên trong lâu đài trước sự chứng_kiến của khoảng 800 khách mời .',
 'Do bố của Meghan vừa trải qua ca mổ tim , Thái_tử Charles , bố của Harry , sẽ là người dẫn cô_dâu vào lễ đường .',
 'Một tiếng sau , họ sẽ cùng lên cỗ xe_ngựa để tham_gia lễ rước quanh toà nhà và chào dân_chúng .',
 '2.640 khách mời là dân_thường do cặp đôi lựa_chọn kỹ_lưỡng sẽ được chứng_kiến nghi_lễ này từ sân lâu đài , trong đó có hàng trăm cá_nhân đến từ các tổ_chức từ_thiện .',
 'Các biện_pháp bảo_vệ an_ninh cho đám_cưới Sau lễ r

In [85]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences[0]).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
nltk.download('stopwords')# one time execution

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
# download pretrained GloVe word embeddings
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-11-17 05:33:24--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-11-17 05:33:24--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-11-17 05:33:24--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

## word to vec

In [21]:
! unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [86]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [87]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [88]:
len(sentence_vectors)

16

## word to vec (vietnamese)

In [25]:
pip install pyvi

     |████████████████████████████████| 8.5 MB 1.6 MB/s 
     |████████████████████████████████| 743 kB 43.9 MB/s 


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [89]:
# Read embedding
word_dict = []
embeddings_index = {}
embedding_dim = 300
max_feature = len(embeddings_index) + 2

f = open('drive/MyDrive/public_dataset/uit-vsfc/W2V_ner.vec')
for line in f:
    values = line.split(' ')
    word = values[0] 
    word_dict.append(word)
    try:
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except Exception as e:
        pass
f.close()

print('Embedding data loaded')

Embedding data loaded


In [90]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([embeddings_index.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((300,))
  sentence_vectors.append(v)

In [91]:
len(sentence_vectors)

16

The next step is to find similarities among the sentences. We will use cosine similarity to find similarity between a pair of sentences. Let's create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

In [92]:
# similarity matrix
sim_mat = np.zeros([len(sentences[0]), len(sentences[0])])

In [93]:
from sklearn.metrics.pairwise import cosine_similarity

In [94]:
for i in range(len(sentences[0])):
  for j in range(len(sentences[0])):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]

In [95]:
len(sentences[0])

16

In [96]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [97]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences[0])), reverse=True)

In [117]:
# Specify number of sentences to form the summary
sn = 1

# Generate summary
list_result = []
str1 = []
for i in range(sn):
  list_result.append(ranked_sentences[i][1])
  str1 = ''.join(str(e) for e in list_result)
  print(ranked_sentences[i][1])

Vào buổi tối , một bữa tiệc cưới mang tính riêng_tư hơn sẽ diễn ra chỉ dành cho 200 người là bạn_bè thân_thiết và gia_đình của cô_dâu , chú_rể tại Frogmore_House .


In [118]:
str1

'Vào buổi tối , một bữa tiệc cưới mang tính riêng_tư hơn sẽ diễn ra chỉ dành cho 200 người là bạn_bè thân_thiết và gia_đình của cô_dâu , chú_rể tại Frogmore_House .'

In [119]:
train_df.summary[0]

'Hoàng tử Anh Harry và nữ diễn viên Mỹ Meghan_Markle hôm_nay sẽ nên vợ nên chồng sau gần hai năm hẹn hò . '

In [106]:
!pip install -q rouge-score

In [120]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(str1,
                      train_df.summary[0])

In [121]:
scores

{'rouge1': Score(precision=0.35294117647058826, recall=0.21818181818181817, fmeasure=0.2696629213483146),
 'rougeL': Score(precision=0.2647058823529412, recall=0.16363636363636364, fmeasure=0.20224719101123595)}